In [2]:
import sys
import pathlib as pl 
tmp = pl.Path(".").absolute() / "code" / "pymicro"
sys.path.append(str(tmp))
print(sys.path)

["d:\\Users\\Valentin\\Desktop\\1A Mines\\Semestre 2\\UE22 - Ingéniérie  logicielle\\ECUE22.3 - Projet d'Informatique", 'c:\\miniconda\\python311.zip', 'c:\\miniconda\\DLLs', 'c:\\miniconda\\Lib', 'c:\\miniconda', '', 'c:\\miniconda\\Lib\\site-packages', 'c:\\miniconda\\Lib\\site-packages\\win32', 'c:\\miniconda\\Lib\\site-packages\\win32\\lib', 'c:\\miniconda\\Lib\\site-packages\\Pythonwin', "d:\\Users\\Valentin\\Desktop\\1A Mines\\Semestre 2\\UE22 - Ingéniérie  logicielle\\ECUE22.3 - Projet d'Informatique\\code\\pymicro"]


In [3]:
from pymicro.crystal.quaternion import Quaternion

In [4]:
import math
import numpy as np

n = 11
eulers = []
phi1 = np.linspace(0.+0.00001, np.pi-0.00001, n, endpoint=True)
Phi = phi1.copy()
phi2 = phi1.copy()
for i in range(len(phi1)):
    for j in range(len(Phi)):
        for k in range(len(phi2)):
            eulers.append([phi1[i], Phi[j], phi2[k]])

eulers = np.array(eulers)
print(eulers)
print(len(eulers))

[[1.00000000e-05 1.00000000e-05 1.00000000e-05]
 [1.00000000e-05 1.00000000e-05 3.14167265e-01]
 [1.00000000e-05 1.00000000e-05 6.28324531e-01]
 ...
 [3.14158265e+00 3.14158265e+00 2.51326812e+00]
 [3.14158265e+00 3.14158265e+00 2.82742539e+00]
 [3.14158265e+00 3.14158265e+00 3.14158265e+00]]
1331


In [5]:
epsilon = np.finfo('float').eps
epsilon

2.220446049250313e-16

In [41]:
def upper_hemishpere_axis(axis):
    if abs(axis[2]) > epsilon:
        if axis[2] < 0.:
            # reverse axis
            return -1. * axis
    elif abs(axis[1]) > epsilon:
        if axis[1] < 0:
            # reverse axis and zero z component
            return np.array([-axis[0], -axis[1], 0.])
    elif axis[0] < 0.:
        # reverse axis and zero y and z components
        return np.array([-axis[0], 0., 0.])
    return axis

def eu2ro_vect(euler):
    """
    Compute the rodrigues vector from the 3 euler angles (in radians).
    :param euler: array of the 3 Euler angles (in radians).
    :return: array of the rodrigues vector as a 3 components numpy array.
    """
    a = 0.5 * (euler[:, 0] - euler[:, 2])
    b = 0.5 * (euler[:, 0] + euler[:, 2])
    r1 = np.tan(0.5 * euler[:, 1]) * np.cos(a) / np.cos(b)
    r2 = np.tan(0.5 * euler[:, 1]) * np.sin(a) / np.cos(b)
    r3 = np.tan(b)
    return np.column_stack((r1, r2, r3))

def eu2om_vect(euler):
    """
    Compute the orientation matrix from the 3 euler angles (in radians).
    :param euler: array of the 3 Euler angles (in radians).
    :return: array of the orientation matrices
    """
    c1 = np.cos(euler[:, 0])
    s1 = np.sin(euler[:, 0])
    c = np.cos(euler[:, 1])
    s = np.sin(euler[:, 1])
    c2 = np.cos(euler[:, 2])
    s2 = np.sin(euler[:, 2])
    # rotation matrix g
    g11 = c1 * c2 - s1 * s2 * c
    g12 = s1 * c2 + c1 * s2 * c
    g13 = s2 * s
    g21 = -c1 * s2 - s1 * c2 * c
    g22 = -s1 * s2 + c1 * c2 * c
    g23 = c2 * s
    g31 = s1 * s
    g32 = -c1 * s
    g33 = c
    g = np.column_stack([g11[:], g12[:], g13[:], g21[:], g22[:], g23[:], g31[:], g32[:], g33[:]]).reshape((len(euler), 3, 3))
    return g

def eu2qu_vect(euler):
    """
    Compute the quaternion from the 3 euler angles (in radians).
    :param euler: array of the 3 euler angles in radians.
    :return: array of `Quaternion` instances representing the rotation.
    """
    P = -1
    phi1 = euler[:, 0]
    Phi = euler[:, 1]
    phi2 = euler[:, 2]
    q0 = np.cos(0.5 * (phi1 + phi2)) * np.cos(0.5 * Phi)
    q1 = np.cos(0.5 * (phi1 - phi2)) * np.sin(0.5 * Phi)
    q2 = np.sin(0.5 * (phi1 - phi2)) * np.sin(0.5 * Phi)
    q3 = np.sin(0.5 * (phi1 + phi2)) * np.cos(0.5 * Phi)
    q = np.array([x/np.sqrt((np.array(x)**2).sum()) for x in zip(q0, q1, q2, q3)])

    # The scalar part must be positive
    negative_indices = np.where(q0 < 0)
    q[negative_indices] = -q[negative_indices]
    """
    # ambiguous rotation
    if q.quat[0] < 3 * epsilon:
        axis = upper_hemishpere_axis(q.quat[1:])
        q.quat = np.array([0., *axis])
    """
    return q

def eu2ax_vect(euler):
    """
    Compute the (axis, angle) representation associated to this (passive)
    rotation expressed by the Euler angles.
    :param euler: 3 euler angles (in radians).
    :returns: a tuple containing the axis (a vector) and the angle (in radians).
    """
    t = np.tan(0.5 * euler[:, 1])
    s = 0.5 * (euler[:, 0] + euler[:, 2])
    d = 0.5 * (euler[:, 0] - euler[:, 2])
    tau = np.sqrt(t ** 2 + np.sin(s) ** 2)
    alpha = 2 * np.arctan2(tau, np.cos(s))
    axis_array = np.zeros((len(euler),3))
    angle_array = np.zeros(len(euler))

    # case 1: alpha > pi
    indices_1 = np.where(alpha > np.pi)
    axis_array[indices_1] = np.column_stack((-t[indices_1] / tau[indices_1] * np.cos(d[indices_1]), -t[indices_1] / tau[indices_1] * np.sin(d[indices_1]), -1 / tau[indices_1] * np.sin(s[indices_1])))
    angle_array[indices_1] = 2 * np.pi - alpha[indices_1]

    #case alpha < pi
    indices_2 = np.where(alpha <= np.pi)
    axis_array[indices_2] = np.column_stack((t[indices_2] / tau[indices_2] * np.cos(d[indices_2]), t[indices_2] / tau[indices_2] * np.sin(d[indices_2]), 1 / tau[indices_2] * np.sin(s[indices_2])))
    angle_array[indices_2] = alpha[indices_2]

    return np.column_stack((axis_array, angle_array))

def ax2om_vect(ax):
    """
    Compute the (passive) orientation matrix associated the rotation
    defined by the given (axis, angle) pair.

    :param axis: the rotation axis.
    :param angle: the rotation angle (radians).
    :returns: the 3x3 orientation matrix.
    """
    c = np.cos(ax[:,3])
    s = np.sin(ax[:,3])

    g11 = c + (1 - c) * ax[:,0] ** 2
    g12 = (1 - c) * ax[:,0] * ax[:,1] + s * ax[:,2]
    g13 = (1 - c) * ax[:,0] * ax[:,2] - s * ax[:,1]
    g21 = (1 - c) * ax[:,0] * ax[:,1] - s * ax[:,2]
    g22 = c + (1 - c) * ax[:,1] ** 2
    g23 = (1 - c) * ax[:,1] * ax[:,2] + s * ax[:,0]
    g31 = (1 - c) * ax[:,0] * ax[:,2] + s * ax[:,1]
    g32 = (1 - c) * ax[:,1] * ax[:,2] - s * ax[:,0]
    g33 = c + (1 - c) * ax[:,2] ** 2

    g = np.column_stack([g11[:], g12[:], g13[:], g21[:], g22[:], g23[:], g31[:], g32[:], g33[:]]).reshape((len(ax), 3, 3))
    return g

def ax2eu(ax):
    return om2eu(ax2om_vect(ax))

def ax2ro(ax):
    if abs(ax[3] - np.pi) < epsilon:
        # handle this case
        pass
    return ax[:3] * np.tan(ax[3] / 2)
    
def ax2qu_vect(ax):
    """
    Compute the quaternion associated the rotation defined by the given
    (axis, angle) pair.

    :param ax: a 4 component vecteur composed by the rotation axis 
        and the rotation angle (radians).
    :return: the corresponding Quaternion.
    """
    q0 = np.cos(0.5 * ax[:,3])
    qqq = np.multiply(ax[:,:3], np.sin(0.5 * ax[:,3])[:, np.newaxis])
    q1, q2, q3 = qqq[:, 0], qqq[:, 1], qqq[:, 2]
    return np.array([x/np.sqrt((np.array(x)**2).sum()) for x in zip(q0, q1, q2, q3)])

def om2eu(g):
    """
    Compute the Euler angles from the orientation matrix.

    This conversion follows the paper of Rowenhorst et al. :cite:`Rowenhorst2015`.
    In particular when :math:`g_{33} = 1` within the machine precision,
    there is no way to determine the values of :math:`\phi_1` and :math:`\phi_2`
    (only their sum is defined). The convention is to attribute
    the entire angle to :math:`\phi_1` and set :math:`\phi_2` to zero.

    :param g: The 3x3 orientation matrix
    :return: The 3 euler angles in radians.
    """
    eps = np.finfo('float').eps
    (phi1, Phi, phi2) = (0.0, 0.0, 0.0)
    # treat special case where g[2, 2] = 1
    if np.abs(g[2, 2]) >= 1 - eps:
        if g[2, 2] > 0.0:
            phi1 = np.arctan2(g[0][1], g[0][0])
        else:
            phi1 = -np.arctan2(-g[0][1], g[0][0])
            Phi = np.pi
    else:
        Phi = np.arccos(g[2][2])
        zeta = 1.0 / np.sqrt(1.0 - g[2][2] ** 2)
        phi1 = np.arctan2(g[2][0] * zeta, -g[2][1] * zeta)
        phi2 = np.arctan2(g[0][2] * zeta, g[1][2] * zeta)
    # ensure angles are in the range [0, 2*pi]
    if phi1 < 0.0:
        phi1 += 2 * np.pi
    if Phi < 0.0:
        Phi += 2 * np.pi
    if phi2 < 0.0:
        phi2 += 2 * np.pi
    return np.array([phi1, Phi, phi2])


def om2ax(om):
    P = -1  # q.convention
    diag_delta = -P * np.array([om[1, 2] - om[2, 1], om[2, 0] - om[0, 2], om[0, 1] - om[1, 0]])
    # make sure cos(omega) is within [-1, 1]
    t = np.clip(0.5 * (np.trace(om) - 1), -1.0, 1.0)
    omega = np.arccos(t)
    if omega < 2 * epsilon:
        return np.array([0.0, 0.0, 1.0, 0.0])
    # determine the right eigenvector corresponding to the eigenvalue of +1
    w, v = np.linalg.eig(om)
    axis = np.real(v.T[np.isclose(w, 1.0 + 0.0j)])[0]
    # check signs, including when diag delta terms are zeros
    axis = np.where(np.abs(diag_delta) < 1e-12, axis, np.abs(axis) * np.sign(diag_delta))
    return np.array([*axis, omega])

def om2ro(om):
    return eu2ro(om2eu(om))

def om2qu_vect(om):
    """
    Compute the quaternion from the orientation matrix

    :param om: Array of orientations matrices
    :returns: Array of quaternion (4-list)
    """
    a11 = om[:, 0, 0]
    a12 = om[:, 0, 1]
    a13 = om[:, 0, 2]
    a21 = om[:, 1, 0]
    a22 = om[:, 1, 1]
    a23 = om[:, 1, 2]
    a31 = om[:, 2, 0]
    a32 = om[:, 2, 1]
    a33 = om[:, 2, 2]

    q0 = 0.5*np.sqrt(1 + a11 + a22 + a33)
    q1 = -0.5*np.sqrt(1 + a11 - a22 - a33)
    q2 = -0.5*np.sqrt(1 - a11 + a22 - a33)
    q3 = -0.5*np.sqrt(1 - a11 - a22 + a33)

    ind1 = np.where(a32 < a23)
    ind2 = np.where(a13 < a31)
    ind3 = np.where(a21 < a12)

    q1[ind1] = -q1[ind1]
    q2[ind2] = -q2[ind2]
    q3[ind3] = -q3[ind3]

    return np.array([x/np.sqrt((np.array(x)**2).sum()) for x in zip(q0, q1, q2, q3)])


def ro2om(rod):
    """
    Compute the orientation matrix from the Rodrigues vector.

    :param rod: The Rodrigues vector as a 3 components array.
    :returns: The 3x3 orientation matrix representing the rotation.
    """
    r = np.linalg.norm(rod)
    I = np.eye(3)
    if r < epsilon:
        # the rodrigues vector is zero, return the identity matrix
        return I
    theta = 2 * np.arctan(r)
    n = rod / r
    omega = np.array([[0.0, n[2], -n[1]],
                      [-n[2], 0.0, n[0]],
                      [n[1], -n[0], 0.0]])
    g = I + np.sin(theta) * omega + (1 - np.cos(theta)) * omega.dot(omega)
    return g

def ro2eu(rod):
    return om2eu(ro2om(rod))

def ro2ax(rod):
    """
    Compute the axis/angle representation from the Rodrigues vector.

    :param rod: The Rodrigues vector as a 3 components array.
    :returns: A tuple in the (axis, angle) form.
    """
    r = np.linalg.norm(rod)
    axis = rod / r
    angle = 2 * np.arctan(r)
    return np.array([*axis, angle])

def ro2qu(rod):
    return ax2qu(ro2ax(rod))

def qu2eu(q):
    P = -1
    (q0, q1, q2, q3) = q
    q03 = q0 ** 2 + q3 ** 2
    q12 = q1 ** 2 + q2 ** 2
    chi = np.sqrt(q03 * q12)
    if chi == 0.:
        if q12 == 0.:
            phi_1 = np.arctan2(-2 * P * q0 * q3, q0 ** 2 - q3 ** 2)
            Phi = 0.
        else:
            phi_1 = np.arctan2(-2 * q1 * q2, q1 ** 2 - q2 ** 2)
            Phi = pi
        phi_2 = 0.
    else:
        phi_1 = np.arctan2((q1 * q3 - P * q0 * q2) / chi,
                      (-P * q0 * q1 - q2 * q3) / chi)
        Phi = np.arctan2(2 * chi, q03 - q12)
        phi_2 = np.arctan2((P * q0 * q2 + q1 * q3) / chi,
                      (q2 * q3 - P * q0 * q1) / chi)
    return np.array([phi_1, Phi, phi_2])
    
def qu2om_vect(q):
    """
    Compute the orientation matrix from the quaternion representation.

    :param q: Array of Quaternion [q0, q1, q2, q3]
    :returns: Array of orientation matrices
    """
    P = -1  # q.convention
    (q0, q1, q2, q3) = q
    qbar = q0 ** 2 - q1 ** 2 - q2 ** 2 - q3 ** 2
    g = np.array([[qbar + 2 * q1 ** 2, 2 * (q1 * q2 - P * q0 * q3), 2 * (q1 * q3 + P * q0 * q2)],
                  [2 * (q1 * q2 + P * q0 * q3), qbar + 2 * q2 ** 2, 2 * (q2 * q3 - P * q0 * q1)],
                  [2 * (q1 * q3 - P * q0 * q2), 2 * (q2 * q3 + P * q0 * q1), qbar + 2 * q3 ** 2]])
    return g

def qu2ax_angle(q):
    q0 = q[:, 0]
    angle = 2*np.arccos(np.clip(q0, epsilon, 1 - epsilon))
    return angle

def qu2ax_axis(q):
    q0, q1, q2, q3 = q[:, 0], q[:, 1], q[:, 2], q[:, 3]
    axis = np.array([x for x in zip(q1, q2, q3)])

    n = len(q0)
    ax_array = np.zeros((n, 3))

    angle0_ind = np.where(q0 >= 1 - epsilon)
    ax_array[angle0_ind] = np.array([[0, 0, 1]]*len(angle0_ind))

    q00_ind = np.where(q0 <= epsilon)
    ax_array[q00_ind] = axis[q00_ind]

    others_ind = np.where(np.logical_and((q0 < 1 - epsilon), (q0 > epsilon)))
    s = np.sign(q0[others_ind])/np.linalg.norm(axis[others_ind], axis=1)
    ax_array[others_ind] = np.multiply(s[:, np.newaxis],axis[others_ind])

    return ax_array

def qu2ro(q):
    s = np.sqrt(q[1] ** 2 + q[2] ** 2 + q[3] ** 2)
    t = np.arctan(q[0])
    return t * np.array([q[1] / s, q[2] / s, q[3] / s])

In [7]:
conversions = [[None, eu2om, eu2ax, eu2ro, eu2qu],
               [om2eu, None, om2ax, om2ro, om2qu],
               [ax2eu, ax2om, None, ax2ro, ax2qu],
               [ro2eu, ro2om, ro2ax, None, ro2qu],
               [qu2eu, qu2om, qu2ax, qu2ro, None]]

def dist(a, b):
    # manhattan distance
    return np.max(np.abs(a - b))

def euler_dist(a, b):
    return dist(eu2qu(a), eu2qu(b))

def om_dist(a, b):
    return dist(a.flatten(), b.flatten())

NameError: name 'eu2om' is not defined

In [ ]:
index = 20
print('input euler', eulers[index])

print('\n* eu2om and om2eu')
#eu2om(eulers[index])
print('output euler', om2eu(eu2om_vect(eulers[index])))
print(dist(eulers[index], om2eu(eu2om_vect(eulers[index]))))
print(euler_dist(eulers[index], om2eu(eu2om_vect(eulers[index]))))

print('\n* eu2ax and ax2eu')
print(eu2ax(eulers[index]))
print('output euler', ax2eu(eu2ax(eulers[index])))
print(euler_dist(eulers[index], ax2eu(eu2ax(eulers[index]))))

print('\n* om2ax and ax2om')
print('input', eu2om_vect(eulers[index]))
print('output', ax2om(om2ax(eu2om_vect(eulers[index]))))
print(om_dist(eu2om_vect(eulers[index]), ax2om(om2ax(eu2om_vect(eulers[index])))))

print('\n* eu2ro and ro2eu')
print(eu2ro_vect(eulers[index]))
print('output euler', ro2eu(eu2ro_vect(eulers[index])))
print(euler_dist(eulers[index], ro2eu(eu2ro_vect(eulers[index]))))
print('with ax repr to compute distance:')
print(eu2ax(eulers[index]))
print('output ax', ro2ax(eu2ro_vect(eulers[index])))
print(dist(eu2ax(eulers[index]), ro2ax(eu2ro_vect(eulers[index]))))

print('\n* ax2ro and ro2ax')
print('input ax', eu2ax(eulers[index]))
print('output ax', ro2ax(ax2ro(eu2ax(eulers[index]))))
print(dist(eu2ax(eulers[index]), ro2ax(ax2ro(eu2ax(eulers[index])))))

print('\n* om2ro and ro2om')
print('input', eu2om_vect(eulers[index]))
print('output', ro2om(om2ro(eu2om_vect(eulers[index]))))
print(om_dist(eu2om_vect(eulers[index]), ro2om(om2ro(eu2om_vect(eulers[index])))))

print('\n* eu2qu and qu2eu')
print(eu2qu_vect(eulers[index]))
print('output euler', qu2eu(eu2qu_vect(eulers[index])))
print(euler_dist(eulers[index], qu2eu(eu2qu_vect(eulers[index]))))

print('\n* om2qu and qu2om')
print('input', eu2om_vect(eulers[index]))
print('output', qu2om(om2qu(eu2om_vect(eulers[index]))))
print(om_dist(eu2om_vect(eulers[index]), qu2om(om2qu(eu2om_vect(eulers[index])))))


input euler [1.00000000e-05 3.14167265e-01 2.82742539e+00]

* om2qu and qu2om
input [[[ 6.123234e-17  1.000000e+00  0.000000e+00]
  [-1.000000e+00  6.123234e-17  0.000000e+00]
  [ 0.000000e+00 -0.000000e+00  1.000000e+00]]]


IndexError: index 1 is out of bounds for axis 0 with size 1

In [ ]:
for i, euler in enumerate(eulers):
    if euler_dist(euler, om2eu(eu2om_vect(euler))) > 1e-6:
        print('error in conversion at index {} with angles {}'.format(i, euler))
    if euler_dist(euler, ax2eu(eu2ax(euler))) > 1e-6:
        print('error in conversion at index {} with angles {}'.format(i, euler))
    if om_dist(eu2om_vect(euler), ax2om(om2ax(eu2om_vect(euler)))) > 1e-6:
        print('error in conversion at index {} with angles {}'.format(i, euler))
    if dist(eu2ax(euler), ro2ax(eu2ro(euler))) > 1e-6:
        print('error in conversion at index {} with angles {}'.format(i, euler))
    if dist(eu2ax(euler), ro2ax(ax2ro(eu2ax(euler)))) > 1e-6:
        print('error in conversion at index {} with angles {}'.format(i, euler))
    if om_dist(eu2om_vect(euler), ro2om(om2ro(eu2om_vect(euler)))) > 1e-6:
        print('error in conversion at index {} with angles {}'.format(i, euler))
    if euler_dist(euler, qu2eu(eu2qu_vect(euler))) > 1e-6:
        print('error in conversion at index {} with angles {}'.format(i, euler))
    #if dist(eu2qu_vect(euler), eu2qu_vect_check(euler)) > 1e-6:
    #    # fix pb with pi rotation
    #    print('error in conversion at index {} with angles {}'.format(i, euler))
    #if om_dist(qu2om(eu2qu_vect(euler)), qu2om_check(eu2qu_vect(euler))) > 1e-6:
    #    print('error in conversion at index {} with angles {}'.format(i, euler))
    if om_dist(eu2om_vect(euler), qu2om(om2qu(eu2om_vect(euler)))) > 1e-6:
        print('error in conversion at index {} with angles {}'.format(i, euler))

NameError: name 'euler_dist' is not defined

In [42]:
qus = eu2qu_vect(eulers)
axs = qu2ax_axis(qus)
axs

array([[ 4.47213596e-01,  0.00000000e+00,  8.94427191e-01],
       [ 3.15669688e-05, -4.99968433e-06,  9.99999999e-01],
       [ 1.53880488e-05, -4.99984612e-06,  1.00000000e+00],
       ...,
       [-9.51057134e-01, -3.09015092e-01, -1.54512301e-06],
       [-9.87688497e-01, -1.56433477e-01, -7.82216771e-07],
       [-1.00000000e+00, -0.00000000e+00, -5.00000000e-11]])

In [34]:
[3, 2, 1]*10


[3,
 2,
 1,
 3,
 2,
 1,
 3,
 2,
 1,
 3,
 2,
 1,
 3,
 2,
 1,
 3,
 2,
 1,
 3,
 2,
 1,
 3,
 2,
 1,
 3,
 2,
 1,
 3,
 2,
 1]